Unduh data korpus Wikipedia Bahasa Indonesia melalui link berikut https://dumps.wikimedia.org/idwiki/latest/. Cari file dengan nama “idwiki-latest-pages-articles.xml.bz2”. Data korpus setiap waktunya mengalami peningkatan. Sekitar agustus 2019 terdapat 406855 kosa kata dengan ukuran 512 MB.

Data korpus tersebut masih menggunakan format XML sehingga perlu di olah ke dalam bentuk teks. library Gensim sudah menyediakan teknik pra poses ini. Untuk mengkonversi format tersebut dapat menggunakan kode berikut:

In [3]:
from __future__ import print_function
 
# Ignore warnings dari gensim
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
 
import logging
import os.path
import sys
 
from gensim.corpora import WikiCorpus
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "idwiki-latest-pages-articles.xml.bz2"
namaFileOutput = "wiki.id.text"
 
space = " "
i = 0
 
# Write file ke variabel namaFileOutput encoder utf-8
output = open(namaFileOutput, 'w', encoding='utf-8')
 
# lower=False: huruf kecil dan besar dibedakan
wiki = WikiCorpus(namaFileInput, lemmatize=None, dictionary={}, lower=False)
for text in wiki.get_texts():
    output.write(' '.join(text) + '\n')
    i = i + 1
    if i % 10000 == 0:
        logger.info("Saved " + str(i) + " articles")
 
output.close()
logger.info("Finished Saved " + str(i) + " articles")

2023-10-25 08:30:23,644: INFO: running C:\Users\ACER\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9013 --control=9011 --hb=9010 --Session.signature_scheme="hmac-sha256" --Session.key=b"c18c6baa-252a-4982-ad6c-3eed609f8f90" --shell=9012 --transport="tcp" --iopub=9014 --f=c:\Users\ACER\AppData\Roaming\jupyter\runtime\kernel-v2-13788cNTDuLAYd29A.json
2023-10-25 08:30:55,996: INFO: Saved 10000 articles
2023-10-25 08:31:15,567: INFO: Saved 20000 articles
2023-10-25 08:31:31,812: INFO: Saved 30000 articles
2023-10-25 08:31:46,292: INFO: Saved 40000 articles
2023-10-25 08:32:02,088: INFO: Saved 50000 articles
2023-10-25 08:32:15,574: INFO: Saved 60000 articles
2023-10-25 08:32:29,100: INFO: Saved 70000 articles
2023-10-25 08:32:43,411: INFO: Saved 80000 articles
2023-10-25 08:33:03,649: INFO: Saved 90000 articles
2023-10-25 08:33:18,498: INFO: Saved 100000 articles
2023-10-25 08:33:32,162: INFO: Saved 110000 articles
2023-10-25 08:33:44,876: INFO

Dari kode di atas akan menghasilkan output file dari rangkaian beberapa artikel, di mana satu barisnya mewakili satu artikel.

Setelah data berhasil terkonversi, selanjutnya dilakukan training Word2Vec menggunakan kode berikut:

In [4]:
import multiprocessing
import logging
import os.path
import sys
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "wiki.id.text"
namaFileOutput = "w2vec_wiki_id300_0.txt"
 
 
# size 300 = 300 dimensi vektor, window 10 = 10 pengaruh kata disekitarnya, min_count 5 = kata-kata yang muncul < 5 kali akan dikeluarkan dari kosakata dan diabaikan selama pelatihan, sg 0 = cbow / sg 1 = skip gram 
model = Word2Vec(LineSentence(namaFileInput), vector_size=300, window=10, min_count=5, sg=0, workers=multiprocessing.cpu_count())
 
# trim unneeded model memory = use (much) less RAM
model.init_sims(replace=True)
model.wv.save_word2vec_format(namaFileOutput, binary=False)

2023-10-25 08:40:47,651: INFO: running C:\Users\ACER\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9013 --control=9011 --hb=9010 --Session.signature_scheme="hmac-sha256" --Session.key=b"c18c6baa-252a-4982-ad6c-3eed609f8f90" --shell=9012 --transport="tcp" --iopub=9014 --f=c:\Users\ACER\AppData\Roaming\jupyter\runtime\kernel-v2-13788cNTDuLAYd29A.json
2023-10-25 08:40:47,656: INFO: collecting all words and their counts
2023-10-25 08:40:47,660: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-10-25 08:40:50,086: INFO: PROGRESS: at sentence #10000, processed 10738098 words, keeping 353874 word types
2023-10-25 08:40:51,676: INFO: PROGRESS: at sentence #20000, processed 17334952 words, keeping 490368 word types
2023-10-25 08:40:52,839: INFO: PROGRESS: at sentence #30000, processed 22148080 words, keeping 591133 word types
2023-10-25 08:40:53,897: INFO: PROGRESS: at sentence #40000, processed 26549341 words, keeping 676

Terdapat beberapa parameter diantaranya:

<b>vector_size</b>: Ukuran dimensi vector yang mewakili setiap token atau kata. Jika memiliki data yang terbatas, nilai size yang digunakan sebaiknya lebih kecil karena akan mempengaruhi kata unik di sekitarnya. Namun, jika memiliki banyak data maka dapat bereksperimen dengan berbagai ukuran.

<b>window</b>: Jarak maksimum antara kata target dengan kata di sekitarnya. Jika nilai window yang digunakan besar, maka terdapat banyak kata terkait disekitarnya (baik di posisi kiri dan kanan dari kata konteks). Secara teori, jika window lebih kecil akan memberikan istilah yang lebih spesifik terhadap suatu konteks kata.

<b>min_count</b>: Frekuensi minimal jumlah kata. Model akan mengabaikan kata-kata yang tidak memenuhi nilai min_count. Kata yang jarang muncul biasanya tidak terlalu penting, jadi lebih baik untuk dihilangkan. Parameter ini mungkin lebih berpengaruh pada effisiensi penggunaan memory dan ukuran file model.

<b>sg</b>: 0 untuk arsitektur Word2Vec CBOW dan 1 untuk arsitektur Word2Vec Skip-gram.

<b>workers</b>: Berapa banyak threads yang digunakan untuk melakukan multiprocessing.

Model yang sudah disimpan dapat dilakukan skenario penggunaannya sebagaimana pada kode di bawah ini untuk melihat kedekatan hubungan antara kata.

In [5]:
import gensim
# import gensim.models.keyedvectors as word2vec
# model = gensim.models.Word2Vec.load(namaFileModel)
# model = word2vec.KeyedVectors.load_word2vec_format(namaFileModel, binary=True)

namaFileModel = "w2vec_wiki_id300_0.txt"

sim = model.wv.most_similar("Jakarta")
print("10 kata terdekat dari Jakarta:{}".format(sim))
sim = model.wv.most_similar("Bandung")
print("10 kata terdekat dari Bandung:{}".format(sim))
 
sim = model.wv.similarity("Yogyakarta", "Surakarta")
print("Kedekatan Yogyakarta-Surakarta: {}".format(sim))
sim = model.wv.similarity("Yogyakarta", "Semarang")
print("Kedekatan Yogyakarta-Semarang: {}".format(sim))
 
sim = model.wv.most_similar_cosmul(positive=['minuman', 'rendang'], negative=['makanan'])
print("makanan-rendang, minuman-?: {}".format(sim))
sim = model.wv.most_similar_cosmul(positive=['mobil', 'honda'], negative=['motor'])
print("motor-honda, mobil-?: {}".format(sim))

10 kata terdekat dari Jakarta:[('Kemang', 0.6203495264053345), ('Warteg', 0.6074423789978027), ('Surabaya', 0.5968841314315796), ('Kemayoran', 0.5820807218551636), ('Cilandak', 0.5615003108978271), ('Warkop', 0.5539660453796387), ('Ciputat', 0.5530072450637817), ('Tebet', 0.5481778383255005), ('Monas', 0.5465117692947388), ('Tomang', 0.544028639793396)]
10 kata terdekat dari Bandung:[('Cimahi', 0.7060422897338867), ('Bogor', 0.6751047372817993), ('Tasikmalaya', 0.6579705476760864), ('Sukabumi', 0.6320185661315918), ('Cianjur', 0.6192764043807983), ('Jatinangor', 0.6187552213668823), ('Garut', 0.6169108748435974), ('Baleendah', 0.610203742980957), ('Purwakarta', 0.6038026213645935), ('Dayeuhkolot', 0.5723363757133484)]
Kedekatan Yogyakarta-Surakarta: 0.7962735891342163
Kedekatan Yogyakarta-Semarang: 0.6563185453414917
makanan-rendang, minuman-?: [('Minuman', 0.8734288457883755), ('kola', 0.8591988801045436), ('Fanta', 0.8513661066409143), ('martabak', 0.8450902562492236), ('Yogurt', 0.8